In [1]:
import numpy as np
from datetime import date, timedelta
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import matplotlib.pyplot as plt
import math
import logging

logging.basicConfig(level='CRITICAL')
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.CRITICAL)
mpl_logger.disabled = True

result = pd.DataFrame()
stage = 0
response = []

oldHeader = {
    "Host": "www1.nseindia.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://www1.nseindia.com/products/content/equities/indices/historical_index_data.htm",
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Methods": "GET,POST,PUT,DELETE,OPTIONS",
    "Access-Control-Allow-Headers": "Content-Type, Access-Control-Allow-Headers, Authorization, X-Requested-With",
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'
}

newheaders = {
            'Connection': 'keep-alive',
            'Cache-Control': 'max-age=0',
            'DNT': '1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36',
            'Sec-Fetch-User': '?1',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-Mode': 'navigate',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
          }

# set stock ticker symbol
symbol = 'SBIN'
series = 'EQ'

# indics = 'NIFTY 50'
indics = 'NIFTY BANK'
 
# set date range for historical prices
end_date = date.today() # 24-03-2021
start_date = end_date - timedelta(days=364) # 24-03-2020
 
# reformat date range
end = end_date.strftime('%d-%m-%Y')
start = start_date.strftime('%d-%m-%Y')

In [2]:
BaseUrl = "https://www.nseindia.com"
url = "https://www.nseindia.com/api/historical/cm/equity?symbol=" + symbol + "&series=[%22" + series + "%22]&from=" + start + "&to=" + end
indicsurl = "https://www1.nseindia.com/products/dynaContent/equities/indices/historicalindices.jsp?indexType=" + indics + "&fromDate=" + start + "&toDate=" + end
indicsOHLCurl = "https://www.nseindia.com/api/equity-stockIndices?index=" + indics
# print(indicsurl)

In [45]:
BaseUrl = "https://www.nseindia.com"
indicsOHLCurl = "https://www.nseindia.com/api/equity-stockIndices?index=" + indics

newheaders = {
            'Connection': 'keep-alive',
            'Cache-Control': 'max-age=0',
            'DNT': '1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36',
            'Sec-Fetch-User': '?1',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-Mode': 'navigate',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
          }

# indics = 'NIFTY 50'
indics = 'NIFTY BANK'

def get_symbol_open_data(indics):
    try:
        output = requests.get(BaseUrl, headers=newheaders)
        s = requests.Session()
        output = s.get(BaseUrl,headers=newheaders)
        output = s.get(indicsOHLCurl, headers=newheaders).json()
    except ValueError:
        print(output, "error")
    return [{'Open' : i['open'], 'previousClose' : i['previousClose'], 'pChange': i['pChange']} for i in output['data'] if indics in i['symbol']]


In [46]:
print(get_symbol_open_data(indics))

[{'Open': 32228.95, 'previousClose': 32275.15, 'pChange': 1.43}]


In [47]:
while(True):
    response = requests.get(indicsurl, headers=oldHeader)
    print(requests.codes.ok)
    if (response.status_code == requests.codes.ok) :
        page_content = BeautifulSoup(response.content, "html.parser")
        try :
            a = page_content.find(id="csvContentDiv").get_text();
            a = a.replace(':', ", \n")
            with open("data.csv", "w") as f :
                f.write(a)
            df = pd.read_csv("data.csv",usecols=['Date','Close'])
            result = pd.concat([result, df])
        except AttributeError :
            break;
    else:
        response.raise_for_status()
    break;

try :
    os.remove("data.csv")
except(OSError) :
    pass

200


In [ ]:
print(result)

In [ ]:
# transform json file to dataframe
# prices = pd.DataFrame(json_prices['data'])[['CH_TIMESTAMP', 'CH_CLOSING_PRICE']]
# prices.rename(columns=['CH_TIMESTAMP':"Date", 'CH_CLOSING_PRICE':"Close"])

prices = pd.DataFrame(result)
# sort dates in descending order
prices['Date'] = pd.to_datetime( prices['Date'] )
prices.sort_values(by=['Date'], inplace=True, ascending=True)
# print(prices)

In [ ]:
# calculate daily logarithmic return
prices['returns'] = (np.log(prices.Close / prices.Close.shift(-1)))

# calculate daily standard deviation of returns
# NumPy's std calculates the population standard deviation by default (it is behaving like Excel's STDEVP).
# To make NumPy's std function behave like Excel's STDEV, pass in the value ddof=1:
daily_std = round(np.std(prices.returns, ddof=1), 6)

# annualized daily standard deviation
samples = len(prices.index)-1
std = daily_std * round(math.sqrt( samples ), 4)
vix = np.round(std*100, 2)

In [ ]:
# Plot histograms
symbol = indics
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
n, bins, patches = ax.hist(prices.returns.values, bins=50, alpha=0.65, color='blue')
 
ax.set_xlabel('log return of stock price')
ax.set_ylabel('frequency of log return')
ax.set_title('Historical Volatility for ' + symbol)
  
# get x and y coordinate limits
x_corr = ax.get_xlim()
y_corr = ax.get_ylim()
  
# make room for text
header = y_corr[1] / 5
y_corr = (y_corr[0], y_corr[1] + header)
ax.set_ylim(y_corr[0], y_corr[1])
 
# print historical volatility on plot
x = x_corr[0] + (x_corr[1] - x_corr[0]) / 30
y = y_corr[1] - (y_corr[1] - y_corr[0]) / 15
ax.text(x, y , 'Annualized Volatility: ' + str(vix)+'%', fontsize=11, fontweight='bold')
x = x_corr[0] + (x_corr[1] - x_corr[0]) / 15
y -= (y_corr[1] - y_corr[0]) / 20
 
# save histogram plot of historical price volatility
fig.tight_layout()
fig.savefig('historical volatility.png')
